<a href="https://colab.research.google.com/github/wTatiya/scrapping_hrms/blob/main/scrapping_hrms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install dependencies
!apt-get update > /dev/null
!apt install chromium-chromedriver > /dev/null
!pip install selenium > /dev/null

import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from tqdm import tqdm

# ✅ Configure browser
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)

# ✅ Step 1: Visit login page
driver.get("https://hrms129.thai-nrls.org/hrms13754/Account/Login")

# ✅ Step 2: Enter credentials
username = "hrmsqsh"
password = "qsh13754"

driver.find_element(By.ID, "txtUserName").send_keys(username)
driver.find_element(By.ID, "txtPass").send_keys(password)
driver.find_element(By.ID, "btnLogon").click()

# ✅ Step 3: Wait and go to data page
time.sleep(2)
driver.get("https://hrms129.thai-nrls.org/hrms13754/Database/RiskBookingAllList")
time.sleep(2)

# ✅ Step 4: Scrape all paginated data (supports both views)
data = []
max_pages = 250  # safeguard for up to 25,000 records

for _ in tqdm(range(max_pages), desc="Scraping pages"):
    time.sleep(1.5)
    rows = driver.find_elements(By.CSS_SELECTOR, "#tbDataList tbody tr")

    if not rows:
        # fallback to div-based card view
        cards = driver.find_elements(By.CSS_SELECTOR, ".ibox-content h3")
        if not cards:
            print("⚠️ No data found on this page.")
            break
        page_text = driver.find_element(By.CSS_SELECTOR, ".ibox-content").text
        data.append([page_text])
        break  # no pagination for this layout

    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if cols:
            data.append([col.text.strip() for col in cols])

    # Try clicking next page
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Next"]')
        if "disabled" in next_btn.get_attribute("class"):
            print("✅ Reached last page.")
            break
        next_btn.click()
    except Exception:
        print("✅ No more pages or next button missing.")
        break


# ✅ Step 5: Convert to CSV
df = pd.DataFrame(data)
df.to_csv("risk_data.csv", index=False)
df.head()



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




Scraping pages:   0%|          | 0/250 [00:11<?, ?it/s]

✅ No more pages or next button missing.


,0,1,2,3,4,5
0,2511000066,GPE301:บุคลากรได้รับภัยคุกคามหรือถูกทำร้ายทางว...,ฝ่ายเวชศาสตร์ชันสูตรงาน Specimen Center,,C,รอยืนยัน\n\nวันที่เกิดเหตุ : 03/11/2025 วันที่...
1,2511000065,CPM201:Medication error : Prescribing (เกิดข้อ...,ห้องจ่ายยาชั้น 6 ตึก 150 ปี,,B,รอยืนยัน\n\nวันที่เกิดเหตุ : 03/11/2025 วันที่...
2,2511000063,CPP204:การสื่อสารหรือส่งต่อข้อมูลการรักษาพยาบา...,หอผู้ป่วยสามัญกุมารเวชกรรม 18D (มอ.4ก),,B,ยืนยันแล้วเป็นอุบัติการณ์ความเสี่ยง/รอแก้ไข\n\...
3,2511000062,CPP204:การสื่อสารหรือส่งต่อข้อมูลการรักษาพยาบา...,หอผู้ป่วยสามัญกุมารเวชกรรม 18D (มอ.4ก),,B,ยืนยันแล้วเป็นอุบัติการณ์ความเสี่ยง/รอแก้ไข\n\...
4,2511000061,CPI204:การไม่ปฏิบัติตามแนวทางป้องกันการแพร่กระ...,หอผู้ป่วยสามัญกุมารเวชกรรม 18D (มอ.4ก),,B,ยืนยันแล้วเป็นอุบัติการณ์ความเสี่ยง/รอแก้ไข\n\...


In [ ]:
from google.colab import files
files.download("risk_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>